# Working on a Object detection Kaggle Dataset (already YOLO formated)

In [ ]:
!pip install ultralytics --upgrade

In [ ]:
import yaml
import torch
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2 as cv

## Fixing the YAML dataset paths to Kaggle input directory

In [ ]:
original_yaml_path = '/kaggle/input/object-detection-wildlife-dataset-yolo-format/final_data/data_wl.yaml'

with open(original_yaml_path, 'r') as f:
    data_dict = yaml.safe_load(f)

base_path = '/kaggle/input/object-detection-wildlife-dataset-yolo-format/final_data'
data_dict['train'] = f'{base_path}/train/images'
data_dict['val'] = f'{base_path}/valid/images'
data_dict['test'] = f'{base_path}/test/images'

fixed_yaml_path = '/kaggle/working/fixed_data.yaml'
with open(fixed_yaml_path, 'w') as f:
    yaml.dump(data_dict, f)

print("Fixed YAML dataset paths saved to:", fixed_yaml_path)

## Setup device

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device_num = 0 if device == 'cuda' else 'cpu'
print(f'Running on device: {device_num}')

## Load & Train model

In [ ]:
model = YOLO('yolov8s.pt')

## Train model
model.train(
    data=fixed_yaml_path,
    epochs=5,
    imgsz=640,
    batch=16,
    device=device_num
)

## Predict on test images

In [ ]:
results = model.predict(
    source=f'{base_path}/test/images',
    conf=0.25,
    save=True
)

## Draw bounding boxes on prediction result

In [ ]:
ind_num = 23
first_result = results[ind_num]
img = cv.imread(first_result.path)
for box in first_result.boxes:
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    cls_id = int(box.cls[0])
    conf = box.conf[0]
    label = f"{first_result.names[cls_id]} {conf:.2f}"
    cv.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv.putText(img, label, (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX,
               0.6, (255, 255, 255), 2)

## Convert BGR to RGB for displaying in matplotlib

In [ ]:
img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
plt.imshow(img_rgb)
plt.axis('off')
plt.show()

## Save model weights

In [ ]:
model.save('animal_detector.pt')
print('Model saved as animal_detector.pt')